In [1]:
import sys
sys.path.append('/home/harish/GitRepos/Smart_Helm/')
import pyxdf
import pandas as pd
from datetime import datetime, timedelta, date
from data_processing.raw_data_parser import transform_eeg

In [2]:
data_streams, header = pyxdf.load_xdf("./test_files/Rytle_KZ_02.xdf")

In [4]:
dt = datetime(year=2022, month=1, day=1)
#eeg_df = transform_eeg("./test_files/Rytle_KZ_02.xdf", dt, "Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf" )

In [10]:
ts = header['info']['datetime'][0]
start_timestamp = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S%z")

In [11]:
start_timestamp

datetime.datetime(2021, 10, 7, 13, 35, 48, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))

In [7]:
#ts = eeg_df["actual_timestamp"][3].to_pydatetime()

In [12]:
def real_timestamps_generator(eeg_df, header):
    """
    to generate  the realtimestamps from reference the header start timestamp and within the length of the lsl_timestamps
    """
    start_timestamp = header['info']['datetime'][0]
    start_lsl_ts = min(eeg_df["lsl_timestamp"])
    start_timestamp_obj = datetime.strptime(start_timestamp, "%Y-%m-%dT%H:%M:%S%z")
    eeg_df["raw_delta_lsl_ts"] = (eeg_df["lsl_timestamp"] - start_lsl_ts) * (10**3)
    eeg_df["delta_lsl_ts"] = pd.to_timedelta(eeg_df["raw_delta_lsl_ts"], unit='ms')
    eeg_df["actual_timestamp"] = start_timestamp_obj + eeg_df["delta_lsl_ts"]
    normalized_ts = eeg_df["actual_timestamp"][0].normalize()
    eeg_df["seconds_of_day"] = (eeg_df["actual_timestamp"] - normalized_ts).dt.total_seconds()
    eeg_df.drop(columns=["delta_lsl_ts", "raw_delta_lsl_ts", "actual_timestamp"], inplace=True)
    return eeg_df

In [13]:
eeg_new = real_timestamps_generator(eeg_df, header)

In [14]:
eeg_new

,lsl_timestamp,eeg_value_1,eeg_value_2,eeg_value_3,eeg_value_4,eeg_value_5,eeg_value_6,eeg_value_7,experiment_date,participant_id,experiment_nr,experiment_location_id,seconds_of_day
0,3568.310771,-9.262370,-18.884218,-7.514693,-12.809883,-9.574642,-13.341015,-3.087368,2022-01-01,KZ,2,Rytle,48948.000
1,3568.314771,-8.861792,-22.709057,-10.004929,-14.683005,-12.207636,-15.105679,-0.407751,2022-01-01,KZ,2,Rytle,48948.004
2,3568.318771,-4.833929,-22.329239,-8.503268,-13.241269,-9.548458,-14.889831,0.804321,2022-01-01,KZ,2,Rytle,48948.008
3,3568.322771,1.891958,-17.686552,-2.411475,-8.603864,-3.070277,-11.810200,0.010910,2022-01-01,KZ,2,Rytle,48948.012
4,3568.326771,8.942053,-10.514809,5.950023,-2.494756,4.253526,-5.733337,-3.287272,2022-01-01,KZ,2,Rytle,48948.016


# calculate the seconds of the day column from the available actual timestamp column

In [5]:
eeg_test = transform_eeg("./test_files/Rytle_KZ_02.xdf", dt, "Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf" )

In [7]:
eeg_test.head()

,lsl_timestamp,eeg_value_1,eeg_value_2,eeg_value_3,eeg_value_4,eeg_value_5,eeg_value_6,eeg_value_7,experiment_date,participant_id,experiment_nr,experiment_location_id,seconds_of_day
0,3568.310771,-9.262370,-18.884218,-7.514693,-12.809883,-9.574642,-13.341015,-3.087368,2022-01-01,KZ,2,Rytle,48948.000
1,3568.314771,-8.861792,-22.709057,-10.004929,-14.683005,-12.207636,-15.105679,-0.407751,2022-01-01,KZ,2,Rytle,48948.004
2,3568.318771,-4.833929,-22.329239,-8.503268,-13.241269,-9.548458,-14.889831,0.804321,2022-01-01,KZ,2,Rytle,48948.008
3,3568.322771,1.891958,-17.686552,-2.411475,-8.603864,-3.070277,-11.810200,0.010910,2022-01-01,KZ,2,Rytle,48948.012
4,3568.326771,8.942053,-10.514809,5.950023,-2.494756,4.253526,-5.733337,-3.287272,2022-01-01,KZ,2,Rytle,48948.016
